In [1]:
import os
import pandas as pd
data = pd.read_csv('data/data.csv', header=None)
data

,0,1,2,3
0,300000,«Наёмники блокчейна» — как ими становятся?,"['Dash', 'блокчейн', 'криптовалюта', 'управлен...",data/habr/post__300000.txt
1,300002,Динамичность должна быть безопасной,"['стартапы', 'программы', 'разработка программ...",data/habr/post__300002.txt
2,300004,Издательства выступают против блокировщика рек...,"['mozilla', 'Brave', 'блокировщики рекламы', '...",data/habr/post__300004.txt
3,300006,"Выход из зоны комфорта, как новая бизнес-модель","['бизнес-модель', 'it', 'стартап', 'интернет',...",data/habr/post__300006.txt
4,300008,«Уберизация» поможет российскому рынку рекрути...,"['mail.ru group', 'headhunter', 'российский ры...",data/habr/post__300008.txt
...,...,...,...,...
177,300390,Капитализация Kamcord превысила $100 млн после...,"['Time Warner', 'time warner cable', 'kamcord'...",data/habr/post__300390.txt
178,300392,Конференция ISDEF-2016 в Казани: почему это бы...,"['isdef', 'конференция', 'ит-бизнес', '\n ...",data/habr/post__300392.txt
179,300394,AVO и другие альтернативы роумингу,"['роуминг', 'voip', 'конкуренты', 'стартапы', ...",data/habr/post__300394.txt
180,300396,Шпаргалка в офис: ключевые метрики для развити...,"['метрики', 'развитие бизнеса', '\n ...",data/habr/post__300396.txt


In [2]:
from utils import read_file
publications = [' '.join((title, read_file(path))) for title, path in zip(data[1], data[3])][:]
joined_publications = ' '.join(publications)

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Предобработки
- Удаление пробельных символов
- Удаление спец символов
- Токенезация
- Удаление стандартного списка стоп-слов
- Удаление чисел
- Приведение к нижнему регистру

In [4]:
import re
from utils import compose
def remove_stopwords(stop_words):
    return lambda tokens: (token for token in tokens if token.lower() not in stop_words)
def tolower(tokens):
    return (token.lower() for token in tokens)
def tokenize(text):
    return word_tokenize(text, language='russian')
def stem(stemmer):
    return lambda tokens: (stemmer.stem(token) for token in tokens)
def remove_numbers(tokens):
    def is_number(string):
        try:
            float(string)
            return True
        except ValueError:
            return False
    return (token for token in tokens if not is_number(token))
def remove_spaces(text):
    return re.sub(r'\s+', ' ', text)
def remove_symbols(symbols, separator=' '):
    return lambda text: re.sub(symbols + '+', separator, text)
def remove_single(tokens):
    return (token for token in tokens if len(token) != 1)

## Специальные символы
- Арифметические операции
- Знаки препинания
- Различные скобки
- Слэши
- "Мягкий" дефис: \xad 
- А также $, #, №, @, €, •

In [5]:
special_symbols = r'''[-+\%*;:\'"()[\]^$#№@!.,?/|\\<>{}—«»€•–]'''
xad_symbol = r'''\xad'''

In [6]:
stemmer = SnowballStemmer('russian')
rus_stem = stem(stemmer)

In [7]:
default_stop_words = set((stemmer.stem(stopword.lower()) for stopword in stopwords.words('russian')))
remove_default_stopwords = remove_stopwords(default_stop_words)

In [8]:
import json
with open('stopwords.json', 'r', encoding='utf-8') as f:
    own_stop_words = set((stemmer.stem(stopword.lower()) for stopword in json.loads(f.read())))
remove_own_stopwords = remove_stopwords(own_stop_words)

In [9]:
preprocesses = [remove_spaces, remove_symbols(special_symbols), remove_symbols(xad_symbol, ''), tokenize, rus_stem, remove_default_stopwords, remove_own_stopwords, remove_single, remove_numbers, tolower]

In [10]:
preprocessed_publications = [' '.join(compose(preprocesses)(publication)) for publication in publications]
joined_preprocessed_publications = ' '.join(preprocessed_publications)

## Меры
- Обычная Евклидова [0, ∞)
- Нормализованная Евклидова \[0, 2]
- Косинусное расстояние \[0, 1]

In [11]:
import scipy as sp
def simple_eucl(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())
def normal_eucl(v1, v2):
    normal_v1 = v1 / sp.linalg.norm(v1.toarray())
    normal_v2 = v2 / sp.linalg.norm(v2.toarray())
    delta = normal_v1 - normal_v2
    return sp.linalg.norm(delta.toarray())
def cosine(v1, v2):
    return sp.spatial.distance.cosine(v1.toarray(), v2.toarray())

In [12]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def make_count(src, min_df=1, max_df=1.5, *, verbose=False, request=None):
    vectorizer = CountVectorizer(min_df=min_df, max_df=max_df)
    x_train = vectorizer.fit_transform(src)
    if verbose:
        sample_count, feature_count = x_train.shape
        print(f'titles_count={sample_count}, words_count={feature_count}')
    if request:
        return vectorizer, x_train, vectorizer.transform([request])
    return vectorizer, x_train
def make_tf_idf(src, *, verbose=False, request=None, top=50):
    vectorizer = TfidfVectorizer()
    x_train = vectorizer.fit_transform(src)
    if verbose:
        idxs = np.argsort(vectorizer.idf_)[::-1]
        features = vectorizer.get_feature_names()
        print(f'titles_count={x_train.shape[0]}, words_count={x_train.shape[1]}')
        print([features[i] for i in idxs[-top:]])
        print([features[i] for i in idxs[:top]])
    if request:
        return vectorizer, x_train, vectorizer.transform([request])
    return vectorizer, x_train

In [13]:
def get_distances(dist_func, src):
    return sorted(((dist_func(x_train.getrow(idx), request_vec), text, idx) for idx, text in enumerate(src)), key=lambda x: x[0])

In [14]:
def check_results(src, count=1):
    total = [(dist, get_distances(dist, src)[:count]) for dist in distances]
    for dist_func, dist_res  in total:
        print(f'-----------{dist_func.__name__}-----------')
        for idx, res in enumerate(dist_res):
            print(f'{idx + 1}. Best publication is {res[2]} with distance = {res[0]} :\n {res[1][:100]}')
    return total

In [15]:
from sklearn.cluster import KMeans
def train(n_clusters=3):
    model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1)
    model.fit(x_train)
    return model
def print_model():
    print('Top terms per cluster:')
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(n_clusters):
        print(f'Cluster {i}:')
        for j in order_centroids[i, :10]:
            print(f' {terms[j]}')

In [16]:
request = 'большие данные'
n_clusters = 3
preprocessed_request = ' '.join(list(compose([tokenize, rus_stem])(request)))
distances = [simple_eucl, normal_eucl, cosine]

In [17]:
vectorizer, x_train, request_vec = make_count(publications, verbose=True, request=request)
results = check_results(publications, 5)

titles_count=182, words_count=26843
-----------simple_eucl-----------
1. Best publication is 25 with distance = 8.0 :
 Мы добавили PayPal для оплаты услуг на «Моём круге» В последнее время участились жалобы зарубежных к
2. Best publication is 11 with distance = 8.366600265340756 :
 Интересные события, произошедшие в выходные Как всегда в понедельник, короткая подборка того, что вы
3. Best publication is 88 with distance = 9.591663046625438 :
 Интересные события, произошедшие в выходные Как всегда в понедельник, короткая подборка того, что вы
4. Best publication is 62 with distance = 9.899494936611665 :
 VLOG программиста о программировании и жизни в Германии Привет!

Вот уже почти два года я живу и раб
5. Best publication is 165 with distance = 12.083045973594572 :
 Интересные события, произошедшие в выходные Как всегда в понедельник, короткая подборка того, что вы
-----------normal_eucl-----------
1. Best publication is 156 with distance = 1.121616326697313 :
 Иллюзия больших данных Д

In [18]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:
 не
 на
 что
 это
 для
 как
 вы
 то
 по
 мы
Cluster 1:
 didi
 lyft
 такси
 cabify
 сервис
 миллионов
 компания
 uber
 также
 на
Cluster 2:
 на
 не
 для
 что
 по
 как
 это
 из
 от
 за


In [19]:
vectorizer, x_train, request_vec = make_tf_idf(publications, verbose=True, request=request)
tfidf_results = check_results(publications, 5)

titles_count=182, words_count=26843
['больше', 'поэтому', 'быть', 'со', 'во', 'этого', 'который', 'вы', 'тем', 'уже', 'того', 'когда', 'также', 'они', 'чем', 'можно', 'том', 'будет', 'он', 'есть', 'время', 'этом', 'же', 'чтобы', 'мы', 'его', 'более', 'при', 'если', 'до', 'может', 'их', 'компании', 'только', 'которые', 'все', 'то', 'или', 'так', 'но', 'это', 'за', 'от', 'как', 'из', 'не', 'по', 'для', 'что', 'на']
['ящиках', 'пассажирский', 'патентными', 'патентные', 'патентному', 'патентной', 'дешевого', 'патенте', 'патентам', 'дешевой', 'дешевом', 'пателя', 'патель', 'пассивный', 'пассивному', 'пассажиры', 'пассажирам', 'децентрализованного', 'пассажира', 'пассажир', 'паспортные', 'деятелей', 'паршивое', 'деятели', 'деятельницей', 'джадд', 'партнёрства', 'партнёрских', 'джастин', 'джез', 'джеймс', 'партнёрам', 'децентрализованной', 'патентование', 'джереми', 'педантично', 'дефицитному', 'пенсионный', 'пенсионного', 'пенсионеры', 'пенсии', 'пенициллин', 'пенг', 'пекки', 'пекка', 'пек',

In [20]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:
 baidu
 video
 cabify
 prime
 kamcord
 подписки
 amazon
 на
 99
 сервис
Cluster 1:
 вещей
 на
 объектов
 бизнес
 для
 интернет
 не
 или
 iot
 что
Cluster 2:
 на
 не
 что
 для
 это
 по
 вы
 как
 мы
 то


In [21]:
vectorizer, x_train, request_vec = make_count(preprocessed_publications, verbose=True, request=preprocessed_request)
preprocessed_results = check_results(preprocessed_publications, 5)

titles_count=182, words_count=11757
-----------simple_eucl-----------
1. Best publication is 25 with distance = 6.855654600401044 :
 добав paypal оплат услуг последн врем участ жалоб зарубежн клиент оплат услуг предлага способ оплат 
2. Best publication is 11 with distance = 7.211102550927978 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни динамичн должн б
3. Best publication is 62 with distance = 7.745966692414834 :
 vlog программист программирован жизн герман привет жив работа гамбург август прошл нача вест влог де
4. Best publication is 88 with distance = 8.660254037844387 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни большинств онлай
5. Best publication is 165 with distance = 10.198039027185569 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни ларр пейдж ежене
-----------normal_eucl-----------
1. Best publication is 156 with distance = 0.6999828727284622 :
 иллюз дан

In [22]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:
 дан
 бизнес
 информац
 аналитик
 решен
 сбор
 информацион
 ресурс
 модел
 инструмент
Cluster 1:
 компан
 бизнес
 пользовател
 работ
 клиент
 нов
 проект
 сайт
 цел
 возможн
Cluster 2:
 код
 характеристик
 безопасн
 обеспечен
 тест
 интеграц
 разработчик
 работа
 непрерывн
 явля


In [23]:
vectorizer, x_train, request_vec = make_tf_idf(preprocessed_publications, verbose=True, request=preprocessed_request)
preprocessed_tfidf_results = check_results(preprocessed_publications, 5)

titles_count=182, words_count=11757
['главн', 'развит', 'сред', 'информац', 'задач', 'интернет', 'основн', 'сайт', 'вид', 'крупн', 'стал', 'цел', 'последн', 'человек', 'количеств', 'необходим', 'сторон', 'след', 'случа', 'дел', 'говор', 'нача', 'времен', 'люд', 'дела', 'люб', 'проект', 'должн', 'важн', 'например', 'вопрос', 'стат', 'явля', 'результат', 'рынк', 'клиент', 'бизнес', 'поэт', 'дан', 'сдела', 'пользовател', 'получ', 'решен', 'част', 'врем', 'работа', 'работ', 'возможн', 'нов', 'компан']
['нашеств', 'контактироват', 'конопл', 'коношенк', 'консерв', 'консист', 'консолидирова', 'констант', 'константин', 'констатац', 'констатирова', 'конструирова', 'конструируем', 'консультацион', 'консультирова', 'контактирова', 'контекстуализац', 'конкретик', 'контрастн', 'контролер', 'контролирован', 'контролл', 'контроллер', 'контур', 'конфигурацион', 'конфигурирован', 'конфиденциальн', 'конфликтн', 'концентрацион', 'концепт', 'конч', 'кончин', 'конкурирует', 'конкретизац', 'кольц', 'компань

In [24]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:
 пользовател
 бизнес
 клиент
 компан
 приложен
 дан
 сайт
 реклам
 нов
 продукт
Cluster 1:
 работ
 цел
 люд
 страх
 жизн
 вопрос
 проблем
 компан
 дел
 врем
Cluster 2:
 компан
 фонд
 российск
 миллиард
 росс
 uber
 рынк
 рубл
 проект
 google


In [25]:
matrix = pd.DataFrame(data={
    'Not Preprocessed': {func.__name__: res[0][0] for func, res in results}, 
    'Preprocessed': {func.__name__: res[0][0] for func, res in preprocessed_results},
    'Not Preprocessed TF-IDF': {func.__name__: res[0][0] for func, res in tfidf_results}, 
    'Preprocessed TF-IDF': {func.__name__: res[0][0] for func, res in preprocessed_tfidf_results},
    }).transpose()
matrix

,simple_eucl,normal_eucl,cosine
Not Preprocessed,8.000000,1.121616,0.629012
Preprocessed,6.855655,0.699983,0.244988
Not Preprocessed TF-IDF,1.103110,1.103110,0.608426
Preprocessed TF-IDF,0.885625,0.885625,0.392166
